In [1]:
!pip install yfinance
!pip install finvizfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.0 MB/s eta 0:00:00


In [1]:
import yfinance as yf
from finvizfinance.quote import finvizfinance
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
import re
import requests


def transform_data_to_ttm(
    info_q: pd.DataFrame,
    info_y: pd.DataFrame
) -> pd.DataFrame:
  """
  Esta función concatena la info anual con la info ttm (ultimos 12 meses)
  La info ttm se calcula a partir de la info trimestral (info_q)

  Arguments
  ---------
  info_q (pd.DataFrame): información trimestral reportada por la empresa
  info_y (pd.DataFrame): información a cierre de los años anteriores reportada por la empresa

  Return
  ------
  info_complete (pd.DataFrame): información concatenada ttm, anual
  """
  # ultimo cierre de año reportado
  last_info_y = info_y.columns.max()

  # yahoo finance no trae el ttm: para sacarlo calculamos la suma de los ultimos q
  # ttm = datetime.now() - relativedelta(years=1)
  # cols_ttm = [x for x in info_q.columns if x > ttm]
  cols_ttm = [x for x in info_q.columns][:4]
  info_ttm = info_q[cols_ttm].sum(axis=1)
  info_ttm = pd.DataFrame(info_ttm).rename(columns={0: "ttm"})

  info_y.columns = [x.strftime("%Y") for x in info_y.columns]

  # concatenacion ttm, info anual
  info_complete = pd.concat([info_ttm, info_y], axis=1)

  # para el estado de resultados,
  # la cantidad de acciones no se suma, porque es una foto
  try:
    info_complete.loc["Basic Average Shares", "ttm"] = info_q.iloc[:,0].loc["Basic Average Shares"]
    info_complete.loc["Diluted Average Shares", "ttm"] = info_q.iloc[:,0].loc["Diluted Average Shares"]
  except Exception as e:
    print(e)

  return info_complete


def validate_kpi_growth(hist_kpis: pd.DataFrame) -> float:
  """
  Esta funcion construye la proporcion de periodods que ha crecido un kpi para una empresa.
  Esta proporcion la multiplica por 10 para obtener un score de 0 a 10.
  Ejemplo: compara las ventas totales de cada año vs el año anterior correspondiente, calculando
  la proporcion de años que se ha presentado crecimiento

  Arguments
  ---------
  hist_kpis (pd.DataFrame): dataframe periodos x kpi

  Return
  ------
  score_ (float): score de 0 a 10
  """
  # validamos que tengamos al menos tres años de analisis
  if hist_kpis.shape[0]>3:
    # indicador a comparar la empresa con su historia
    results = []
    for kpi in hist_kpis.columns:
      print(f"Analizando kpi: {kpi}")

      # Numero de periodos que el kpi ha crecido
      num_per_growth = sum(hist_kpis[kpi] > hist_kpis[kpi].shift(-1))

      # proporcion de periodos que el kpi ha crecido
      # restamos 1 al denominador porque el primer periodo
      # no lo podemos comparar con un periodo anterior
      prop_per_growth = num_per_growth / (hist_kpis.shape[0]-1)

      # concatenamos para cada kpi, la proporcion de periodos que crecio
      results.append(prop_per_growth)

    # sacamos score del 0 al 10, dandole el mismo peso a cada kpi
    score_ = 10*sum(results)/len(results)
  else:
    score_ = None
    print("Sin información suficiente para analizar la empresa")

  print("=="*20)
  print(f"Score de crecimiento: {score_}")

  return score_

In [2]:
# ticker_interes = "UBER"
ticker_interes = "PEP"
# ticker_interes = "MSFT"
# ticker_interes = "BABA"
# ticker_interes = "NU"

In [3]:
tick = yf.Ticker(ticker_interes)

# Analisis del estado de resultados de la empresa

In [4]:
def get_statement_results_ticker(tick):

  income_stmt_complete = transform_data_to_ttm(
      info_q=tick.quarterly_income_stmt,
      info_y=tick.income_stmt
  )
  # indicadores para revisar que la empresa se encuentre creciendo
  ix_interes = [
      "Total Revenue",
      "Gross Profit", # - cost of revenue (quitarle el costo de venta)
      "Operating Income", # - operating expense (quitarle los gastos operativos)
      "Net Income" # - (quitarle intereses e impuestos)
  ]
  income_stmt = income_stmt_complete.copy().loc[ix_interes,:].T
  income_stmt = income_stmt.dropna()

  return income_stmt_complete, income_stmt

In [5]:
income_stmt_complete, income_stmt = get_statement_results_ticker(tick)
income_stmt/1000000

,Total Revenue,Gross Profit,Operating Income,Net Income
ttm,91920.0,50260.0,13241.0,9357.0
2023,91471.0,49590.0,12913.0,9074.0
2022,86392.0,45816.0,11357.0,8910.0
2021,79474.0,42399.0,11162.0,7618.0
2020,70372.0,38575.0,10122.0,7120.0


### Vamos a revisar que la empresa se encuentre creciendo

In [159]:
score_stmt_res_growth = validate_kpi_growth(income_stmt)

Analizando kpi: Total Revenue
Analizando kpi: Gross Profit
Analizando kpi: Operating Income
Analizando kpi: Net Income
Score de crecimiento: 10.0


### Vamos a revisar versus la competencia

In [160]:
def get_summary_margins_ttm(inc_stmt: pd.DataFrame):

  per_compare = inc_stmt.loc["ttm", :]
  margins = {
      "Gross M": per_compare["Gross Profit"] / per_compare["Total Revenue"],
      "Oper M": per_compare["Operating Income"] / per_compare["Total Revenue"],
      "Profit M": per_compare["Net Income"] / per_compare["Total Revenue"],
  }
  return margins

In [161]:
# margenes de la empresa de interes
print(f"Margenes de la empresa de interes: {ticker_interes}\n")
margin_interes = get_summary_margins_ttm(income_stmt)
margin_interes

Margenes de la empresa de interes: PEP



{'Gross M': 0.5467798085291558,
 'Oper M': 0.1440491731940818,
 'Profit M': 0.10179503916449086}

In [162]:
# Ticker de interes
stock = finvizfinance(ticker_interes)

# Obtencion de los competidores segun FinViz
# basado en la estructura html de la pagina
attrs = {
    "class": "tab-link",
    "href": re.compile("^screener")
}
peers = [
    x for x in stock.soup.findAll("a", attrs)
    if "Peers" in x
][0]
peers = peers.get("href").split("=")[1].split(",")

# primeros k competidores
k = 5
peers = peers[:k]
peers

['KO', 'KDP', 'MNST', 'FIZZ', 'PRMW']

In [163]:
peers_margin = {}
beat_peers = []
for peer in peers:
  print(peer)
  try:
    tick_peer = yf.Ticker(peer)
    _, income_stmt_peer = get_statement_results_ticker(tick_peer)
    peer_margin = get_summary_margins_ttm(income_stmt_peer)

    for kpi in peer_margin.keys():
      if margin_interes[kpi]>peer_margin[kpi]:
        beat_peers.append(1)
      else:
        beat_peers.append(0)

    peers_margin[peer] = peer_margin
  except Exception as e:
    print(e)

KO
KDP
MNST
FIZZ
PRMW


In [164]:
peers_margin

{'KO': {'Gross M': 0.6043437001250916,
  'Oper M': 0.29154984255704613,
  'Profit M': 0.2244963982228357},
 'KDP': {'Gross M': 0.5569052020068656,
  'Oper M': 0.22913916028518616,
  'Profit M': 0.15038288883020862},
 'MNST': {'Gross M': 0.5373492840548342,
  'Oper M': 0.2726355368813611,
  'Profit M': 0.22842263788597383},
 'FIZZ': {'Gross M': 0.36460201833528694,
  'Oper M': 0.18791455117981298,
  'Profit M': 0.15362758129777337},
 'PRMW': {'Gross M': 0.6690248565965583,
  'Oper M': 0.11542383683875079,
  'Profit M': 0.16061185468451242}}

In [165]:
score_stmt_res_peers = sum(beat_peers)*10/len(beat_peers)
score_stmt_res_peers

2.0

In [166]:
score_stmt_res = score_stmt_res_growth*0.6 + score_stmt_res_peers*0.4
score_stmt_res

6.8

In [167]:
# dudas: el margen operacional no me coincidio en ninguna pagina, como sera que lo estan tomando?

# Balance General

In [168]:
tick = yf.Ticker(ticker_interes)

balance_general = tick.quarterly_balance_sheet
print(balance_general.shape)
balance_general.head(3)

(79, 7)


,2024-09-30,2024-06-30,2024-03-31,2023-12-31,2023-09-30,2023-06-30,2023-03-31
Treasury Shares Number,494000000.0,493000000.0,492000000.0,493000000.0,492000000.0,NaN,NaN
Ordinary Shares Number,1371989025.0,1373572400.0,1375000000.0,1374000000.0,1374863961.0,NaN,NaN
Share Issued,1865989025.0,1866572400.0,1867000000.0,1867000000.0,1866863961.0,NaN,NaN


In [169]:
# Dinero en caja que cubra mas de tres meses de operación
# Total cash and short term investments > selling general & admin expenses, total

# gastos totales de operacion
operation_expenses_month = income_stmt_complete["ttm"]["Selling General And Administration"] / 12
print(f"Gastos operativos mensuales: {round(operation_expenses_month)}")

# dinero en caja
cash = balance_general.iloc[:,0]["Cash Cash Equivalents And Short Term Investments"]
print(f"Dinero en caja: {cash}")

# meses de operacion que se cubren con la caja
months_operation = round(cash / operation_expenses_month, 2)
months_operation

Gastos operativos mensuales: 3084916667
Dinero en caja: 8051000000.0


2.61

In [170]:
# activos corrientes
current_assets = balance_general.iloc[:,0]["Current Assets"]

# pasivos corrientes
current_liabili = balance_general.iloc[:,0]["Current Liabilities"]

# current ratio>0.7?
current_ratio = round(current_assets/current_liabili, 2)
print(f"Current ratio: {current_ratio}")


# Inventario
try:
  inventory = balance_general.iloc[:,0]["Inventory"]
  # quickRatio>0.7?
  quick_ratio = round((current_assets-inventory)/current_liabili, 2)
except Exception as e:
  quick_ratio = None
  print(e)

print(f"Quick ratio: {quick_ratio}")

Current ratio: 0.89
Quick ratio: 0.7


In [171]:
# Total Debt
total_debt = balance_general.iloc[:,0]["Total Debt"]

# total Assets
total_assets = balance_general.iloc[:,0]["Total Assets"]

# Total Debt / Total Assets
debt_ratio = round(total_debt/total_assets, 2)
print(f"Debt ratio: {debt_ratio}")

Debt ratio: 0.45


In [172]:
score_balance = []
kpis_rules = {
    "months_operation": {"var": months_operation, "val": 3},
    "current_ratio": {"var": current_ratio, "val": 0.7},
    "quick_ratio": {"var": quick_ratio, "val": 0.7},
    "debt_ratio": {"var": debt_ratio, "val": 0.5}
}
for kpiname, info_dict in kpis_rules.items():
  print(kpiname)
  kpi = info_dict["var"]
  rule = info_dict["val"]
  if kpi:
    if kpiname in ["debt_ratio"]:
      score_balance += [1] if kpi<=rule else [0]
    else:
      score_balance += [1] if kpi>=rule else [0]

score_balance = 10*sum(score_balance)/len(score_balance)
print(f"Calificacion balance general: {score_balance}")

months_operation
current_ratio
quick_ratio
debt_ratio
Calificacion balance general: 7.5


# Flujo de caja

In [173]:
cash_flow_complete = transform_data_to_ttm(
    info_q=tick.quarterly_cash_flow,
    info_y=tick.cash_flow
)
# indicadores para revisar que la empresa se encuentre creciendo
ix_interes = [
    "Operating Cash Flow",
    "Free Cash Flow"
]
cash_flow_filtered = cash_flow_complete.copy().loc[ix_interes,:].T
cash_flow_filtered = cash_flow_filtered.dropna()

'Basic Average Shares'


In [174]:
cash_flow_filtered

,Operating Cash Flow,Free Cash Flow
ttm,12032000000.0,6201000000.0
2023,13442000000.0,7924000000.0
2022,10811000000.0,5604000000.0
2021,11616000000.0,6991000000.0
2020,10613000000.0,6373000000.0


In [175]:
score_cash_flow = validate_kpi_growth(cash_flow_filtered)

Analizando kpi: Operating Cash Flow
Analizando kpi: Free Cash Flow
Score de crecimiento: 5.0


In [176]:
score_financial_final = (1.*score_stmt_res + 1.*score_balance + 1.*score_cash_flow)/3
score_financial_final = round(score_financial_final, 2)
print(f"Score salud financiera final: {score_financial_final}")

Score salud financiera final: 6.43


---

# Precio

In [177]:
def get_historic_price(tick):
  """Funcion que trae el precio historico de una accion dada"""

  hist_price = tick.history(period="5Y")
  hist_price.index = [x.strftime("%Y-%m-%d") for x in hist_price.index]
  hist_price = hist_price[["Close", "Volume"]]
  print(hist_price.index.min(), ";", hist_price.index.max())
  hist_price = hist_price.reset_index()
  hist_price = hist_price.rename(columns={"index": "period"})

  return hist_price

In [178]:
# precio historico de la accion en analisis
hist_price_interes = get_historic_price(tick)
hist_price_interes.tail()

2019-11-08 ; 2024-11-07


,period,Close,Volume
1253,2024-11-01,165.589996,6339000
1254,2024-11-04,166.339996,4885500
1255,2024-11-05,167.850006,3678300
1256,2024-11-06,164.710007,7849300
1257,2024-11-07,164.000000,5020767


In [179]:
def request_historic_ttm_data(data_type: str, ticker: str):
  """Funcion para traerse la informacion financiera historica TTM de la accion"""

  # Nos traemos los estados de resultados TTM... TTM data
  if data_type=="income":
    url = f"https://stockanalysis.com/stocks/{ticker.lower()}/financials/?p=trailing"
  elif data_type=="cash_flow":
    url = f"https://stockanalysis.com/stocks/{ticker.lower()}/financials/cash-flow-statement/?p=trailing"

  print(url)
  response = requests.get(url)
  if response.status_code==200:
    print("Request successful")

  return response.content

In [180]:
def get_historic_ttm(data_type: str, ticker: str):
  """Funcion para parsear o formatear la info financiera TTM"""

  # extraer de la pagina stockanalysis la tabla ttm historico
  response_income = request_historic_ttm_data(data_type=data_type, ticker=ticker)
  hist_ttm = pd.read_html(response_income)[0]
  hist_ttm.columns = [x[1][8:].strip() for x in hist_ttm.columns]

  # filtrado de la tabla para quedarnos con los indicadores de interes
  if data_type=="income":
    kpis = ["Revenue", "Gross Profit", "Net Income", "Shares Outstanding (Basic)"]
  elif data_type=="cash_flow":
    kpis = ["Free Cash Flow"]
  hist_ttm = hist_ttm[hist_ttm.iloc[:,0].isin(kpis)].T
  hist_ttm.columns = hist_ttm.iloc[0, :]
  hist_ttm = hist_ttm.iloc[1:,:]
  hist_ttm = hist_ttm.reset_index()
  hist_ttm = hist_ttm.rename(columns={"index": "period"})
  hist_ttm = hist_ttm[hist_ttm[kpis[0]]!="Upgrade"]
  hist_ttm["earning_date"] = pd.to_datetime(
    hist_ttm["period"], format="%b %d, %Y"
  ).dt.strftime("%Y-%m-%d")
  hist_ttm = hist_ttm.drop(columns=["period"])

  # casteo de los indicadores porque vienen tipo object
  for x in kpis:
    hist_ttm[x] = hist_ttm[x].astype(float)

  return hist_ttm

In [181]:
historic_income_ttm_interes = get_historic_ttm(data_type="income", ticker=ticker_interes)

print(historic_income_ttm_interes.shape)
historic_income_ttm_interes.head()

https://stockanalysis.com/stocks/pep/financials/?p=trailing
Request successful
(20, 5)


nding,Revenue,Gross Profit,Net Income,Shares Outstanding (Basic),earning_date
0,91920.0,50448.0,9357.0,1374.0,2024-09-07
1,92054.0,50303.0,9519.0,1375.0,2024-06-15
2,91875.0,49750.0,9184.0,1375.0,2024-03-23
3,91471.0,49603.0,9074.0,1376.0,2023-12-30
4,91617.0,49491.0,8290.0,1376.0,2023-09-09


In [182]:
historic_cash_ttm_interes = get_historic_ttm(data_type="cash_flow", ticker=ticker_interes)

print(historic_cash_ttm_interes.shape)
historic_cash_ttm_interes.head()

https://stockanalysis.com/stocks/pep/financials/cash-flow-statement/?p=trailing
Request successful
(20, 2)


nding,Free Cash Flow,earning_date
0,6201.0,2024-09-07
1,7032.0,2024-06-15
2,7242.0,2024-03-23
3,7924.0,2023-12-30
4,6947.0,2023-09-09


In [183]:
# income ttm historico + cash ttm historico
historic_ttm_interes = historic_income_ttm_interes.merge(
    historic_cash_ttm_interes,
    on=["earning_date"]
)
historic_ttm_interes = historic_ttm_interes.sort_values(["earning_date"])
print(historic_ttm_interes.shape)
historic_ttm_interes.head()

(20, 6)


nding,Revenue,Gross Profit,Net Income,Shares Outstanding (Basic),earning_date,Free Cash Flow
19,67161.0,37178.0,7314.0,1399.0,2019-12-28,5417.0
18,68158.0,37730.0,7239.0,1395.0,2020-03-21,4971.0
17,67654.0,37446.0,6850.0,1392.0,2020-06-13,5470.0
16,68557.0,37889.0,7041.0,1389.0,2020-09-05,6362.0
15,70372.0,38637.0,7120.0,1385.0,2020-12-26,6373.0


In [184]:
# si la moneda es diferente a usd
earnings_currency = tick.info["financialCurrency"]
if earnings_currency!="USD":
  tick_currency = yf.Ticker(f"{earnings_currency}=X")
  exchange_rate = tick_currency.history(period="5Y")[["Close"]]
  exchange_rate = exchange_rate.reset_index()
  exchange_rate.columns = ["period", "exchange"]
  exchange_rate["period"] = exchange_rate["period"].dt.strftime("%Y-%m-%d")
  exchange_rate["exchange"] = exchange_rate["exchange"].astype(float)
  exchange_rate = exchange_rate.sort_values(["period"])

  # cruce con el precio de la accion para dejar el precio en moneda local
  # o la misma moneda de los estados financieros, ya que el precio
  # por defecto viene en usd
  hist_price_adj = hist_price_interes.merge(exchange_rate, on=["period"], how="left")
  hist_price_adj["close_adj_currency"] = hist_price_adj["Close"]*hist_price_adj["exchange"]
else:
  hist_price_adj = hist_price_interes.copy()
  hist_price_adj["close_adj_currency"] = hist_price_adj["Close"].copy()

In [185]:
hist_ttm_replicated = []

for i in range(historic_ttm_interes.shape[0]):

  row_i = historic_ttm_interes.iloc[i]
  try:
    row_iplus1 = historic_ttm_interes.iloc[i+1]
    ends = row_iplus1["earning_date"]
    inclusive = "left"
  except Exception as e:
    ends = datetime.now().strftime("%Y-%m-%d")
    inclusive = "both"

  dts = pd.date_range(
      row_i["earning_date"],
      ends,
      inclusive=inclusive
  )
  dts = dts.strftime("%Y-%m-%d")
  replicated_i = pd.DataFrame(dts, columns=["period"])
  for var, val in row_i.items():
    replicated_i[var] = val

  hist_ttm_replicated.append(replicated_i)

hist_ttm_replicated = pd.concat(hist_ttm_replicated)
print(hist_ttm_replicated.shape)
hist_ttm_replicated.head(3)

(1778, 7)


,period,Revenue,Gross Profit,Net Income,Shares Outstanding (Basic),earning_date,Free Cash Flow
0,2019-12-28,67161.0,37178.0,7314.0,1399.0,2019-12-28,5417.0
1,2019-12-29,67161.0,37178.0,7314.0,1399.0,2019-12-28,5417.0
2,2019-12-30,67161.0,37178.0,7314.0,1399.0,2019-12-28,5417.0


In [186]:
print(hist_price_adj.shape)
hist_price_kpis = hist_price_adj.merge(
    hist_ttm_replicated,
    on=["period"], how="left"
)
print(hist_price_kpis.shape)
hist_price_kpis.head()

(1258, 4)
(1258, 10)


,period,Close,Volume,close_adj_currency,Revenue,Gross Profit,Net Income,Shares Outstanding (Basic),earning_date,Free Cash Flow
0,2019-11-08,115.477364,2604000,115.477364,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-11-11,114.835480,2227200,114.835480,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-11-12,114.479858,3246500,114.479858,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-11-13,115.616150,3166600,115.616150,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-11-14,115.650833,2141900,115.650833,NaN,NaN,NaN,NaN,NaN,NaN


In [187]:
# calculo de ratios
price_var = "close_adj_currency"
shares_var = "Shares Outstanding (Basic)"
hist_price_kpis["pe_ratio"] = (
    hist_price_kpis[price_var] * hist_price_kpis[shares_var] / hist_price_kpis["Net Income"]
)
hist_price_kpis["ps_ratio"] = (
    hist_price_kpis[price_var] * hist_price_kpis[shares_var] / hist_price_kpis["Revenue"]
)
hist_price_kpis["pgp_ratio"] = (
    hist_price_kpis[price_var] * hist_price_kpis[shares_var] / hist_price_kpis["Gross Profit"]
)
hist_price_kpis["pfcf_ratio"] = (
    hist_price_kpis[price_var] * hist_price_kpis[shares_var] / hist_price_kpis["Free Cash Flow"]
)

In [188]:
score_precio_hist = []
for multiplo in ["pe_ratio", "ps_ratio", "pgp_ratio", "pfcf_ratio"]:
  print("============")
  print(multiplo)
  last_val = hist_price_kpis[multiplo].iloc[-1]

  max_hist = hist_price_kpis[multiplo].max()
  min_hist = hist_price_kpis[multiplo].min()

  print(last_val)
  print(min_hist, max_hist)

  score_multiplo = (max_hist-last_val)/(max_hist-min_hist)
  # print("score_multiplo", score_multiplo)
  score_precio_hist.append(score_multiplo)

score_precio_hist = round( 10*sum(score_precio_hist)/len(score_precio_hist), 2 )

pe_ratio
24.082077588970822
17.481492531504955 39.30250123619145
ps_ratio
2.4514360313315926
1.8930642666882893 2.937933172441173
pgp_ratio
4.466698382492864
3.4197581311672525 5.503303488060937
pfcf_ratio
36.33865505563619
23.603403429098623 48.554488574314995


In [189]:
score_precio_hist

5.38

# Comparando el precio con la competencia

In [190]:
peers

['KO', 'KDP', 'MNST', 'FIZZ', 'PRMW']

---

In [ ]:
tech = yf.Sector('technology')

In [ ]:
# tech.key
# tech.name
# tech.symbol
# tech.ticker
# tech.overview
# tech.top_companies
# tech.research_reports

In [ ]:
# # Sector and Industry to Ticker
# tech_ticker = tech.ticker
# tech_ticker.info